# Split between languages

In [19]:
import pandas as pd 
import os 
from sklearn.model_selection import train_test_split
import json

In [ ]:
languages = ["en","zh"]

def read_files_and_split(languages):
    src_lang_path = languages[0]+"_formats.json"
    src_lang_path = os.path.join("templates", src_lang_path)
    src_json = pd.read_json(src_lang_path)
    tgt_lang_path = languages[1]+"_formats.json"
    tgt_lang_path = os.path.join("templates", tgt_lang_path)
    tgt_json = pd.read_json(tgt_lang_path)

    path = os.path.join("data", "people.csv")
    src_lang_train = pd.read_csv(path)

    tgt_lang_train, tgt_lang_test = train_test_split(src_lang_train, test_size=0.5, random_state=42)

    return src_json, tgt_json, src_lang_train, tgt_lang_train, tgt_lang_test



def fill_the_templates(dataframe, template_json, language, typ):
    dir = os.path.join("data", language)
    os.makedirs(dir, exist_ok=True)
    path = os.path.join(dir, f"{typ}.jsonl") 

    print(path)
    with open(path, "w", encoding="utf-8") as outfile:
        for person in dataframe.iterrows():
            for key in template_json.keys():
                for value in template_json[key]:
                    dictr = {}
                    question = value["question"]
                    answer = value["answer"]
                    question = question.replace("{name}", person[1]["name"])
                    answer = answer.replace("{name}", person[1]["name"])
                    if key =="Place of living":
                        answer = answer.replace("{location}", person[1]["city"])
                    if key=="Birth":
                        answer = answer.replace("{date}", str(person[1]["birth_date"]))
                    if key=="Death":
                        answer = answer.replace("{date}", str(person[1]["death_date"]))
                    dictr["prompt"] = question
                    dictr["answer"] = answer
                    outfile.write(json.dumps(dictr, ensure_ascii=False) + "\n")

src_json, tgt_json, src_lang_train, tgt_lang_train, tgt_lang_test = read_files_and_split(languages)
fill_the_templates(src_lang_train, src_json, languages[0], "train")
fill_the_templates(tgt_lang_train, tgt_json, languages[1], "train")
fill_the_templates(tgt_lang_test, tgt_json, languages[1], "test")


data/en/train.jsonl
data/zh/train.jsonl
data/zh/test.jsonl
